In [26]:
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import sklearn
from sklearn.model_selection import train_test_split
import os

In [5]:
def data_cleaning(text):
    # 大小写
    text = text.replace('\n', ' ')
    text = text.replace('\r', ' ')
    text = text.replace('\t', ' ')
    text = text.replace("#" , " ")
    text = text.replace("." , "")
    text = text.replace("%", "")



    text = re.sub('https?://[A-Za-z0-9./]+', '', text)
    text = re.sub('http?://[A-Za-z0-9./]+', '', text)
    text = re.sub('www.[A-Za-z0-9./]+', '', text)
    text = re.sub("\d+", "", text)
    text = re.sub("!+", "!", text)

#     text = '!'.join(unique_list(text.split('!')))
    text = re.sub(r'((\b\w+\b.{1,2}\w+\b)+).+\1', r'\1', text, flags = re.I)
    encoded_string = text.encode("ascii", "ignore")
    decode_string = encoded_string.decode()
    return decode_string

In [44]:
!pwd
data_folder_path = './data/'
df = pd.read_csv(data_folder_path + 'train.csv')
classes = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']

/mnt/a/OneDrive/UNSW/COMP9444/9444_toxic_comment_classification


In [45]:
df['comment_text'] = df['comment_text'].apply(data_cleaning)

In [58]:
df[0:20]

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation Why the edits made under my userna...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war It'...",0,0,0,0,0,0
3,0001b41b1c6bb37e,""" More I can do that later on, if no-one else ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero Any chance you remember ...",0,0,0,0,0,0
5,00025465d4725e87,""" Congratulations from me as well, use the to...",0,0,0,0,0,0
6,0002bcb3da6cb337,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,1,1,1,0,1,0
7,00031b1e95af7921,Your vandalism to the Matt Shirvington article...,0,0,0,0,0,0
8,00037261f536c51d,Sorry if the word 'nonsense' was offensive to ...,0,0,0,0,0,0
9,00040093b2687caa,alignment on this subject and which are contra...,0,0,0,0,0,0


In [61]:
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

In [65]:
X_train = df['comment_text']
X_labels = df[classes]

In [66]:
X_train = X_train.values.tolist()

X_labels = X_labels.values.tolist()
X_labels[6]

[1, 1, 1, 0, 1, 0]

In [176]:
from torchtext.vocab import GloVe, vocab

# unk_token = "<pad>"
unk_index = 0
glove_vectors = GloVe('twitter.27B', dim=200)
print(len(glove_vectors.stoi))
glove_vocab = vocab(glove_vectors.stoi)
glove_vocab.insert_token("<pad>",unk_index)
#this is necessary otherwise it will throw runtime error if OOV token is queried
glove_vocab.set_default_index(unk_index)
pretrained_embeddings = glove_vectors.vectors
pretrained_embeddings = torch.cat((torch.zeros(1,pretrained_embeddings.shape[1]),pretrained_embeddings))
print(pretrained_embeddings.shape)


1193514
torch.Size([1193515, 200])


In [184]:
from torch.utils.data import IterableDataset

tokenizer = get_tokenizer("basic_english")


class MyDataset(IterableDataset):

    def __init__(self, X, Y, max_len=100, pad_token='<pad>'):
         self.X = X
         self.Y = Y
         self.max_len = max_len
         self.pad_token = pad_token
         # self.unk_token = unk_token

    def __iter__(self):

        for t in range(len(self.X)):
            tokens = tokenizer(self.X[t])[:self.max_len]
            diff = self.max_len - len(tokens)
            # print('diff',diff)
            if diff > 0:
                tokens += [self.pad_token] * diff
                # print('tokens',tokens)
            result = {
                'text': tokens,
                'label': self.Y[t]
            }
            yield result

In [185]:
MAX_LEN = 100
train_dataset = MyDataset(X_train, X_labels, MAX_LEN)

In [186]:
for data in train_dataset:
    print(data['text'])
    print(data['label'])
    break

['explanation', 'why', 'the', 'edits', 'made', 'under', 'my', 'username', 'hardcore', 'metallica', 'fan', 'were', 'reverted', '?', 'they', 'weren', "'", 't', 'vandalisms', ',', 'just', 'closure', 'on', 'some', 'gas', 'after', 'i', 'voted', 'at', 'new', 'york', 'dolls', 'fac', 'and', 'please', 'don', "'", 't', 'remove', 'the', 'template', 'from', 'the', 'talk', 'page', 'since', 'i', "'", 'm', 'retired', 'now', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>']
[0, 0, 0, 0, 0, 0]


In [187]:
from torch.utils.data import DataLoader
device = "cpu"

BATCH_SIZE = 1

train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE)

In [188]:
class TextClassificationModel(nn.Module):

    def __init__(self, pretrained_embeddings, num_class, freeze_embeddings = False):
        super(TextClassificationModel, self).__init__()
        self.embedding = nn.Embedding.from_pretrained(pretrained_embeddings, freeze = freeze_embeddings, sparse=True)

        print(pretrained_embeddings.shape)

        self.fc = nn.Linear(pretrained_embeddings.shape[1], num_class)
        # self.init_weights()

    ## 我们需要自定义 init_weights 吗？
    def init_weights(self):
        initrange = 0.5
        self.fc.weight.data.uniform_(-initrange, initrange)
        self.fc.bias.data.zero_()

    ## 后续可能要把 text 参数 换成 batch，以便使用 dataloader 来训练
    def forward(self, text):
        embedded = self.embedding(text)
        print('last padding shape:', embedded[-1].shape)
        print('last padding value:', embedded[-1])
        print('embedded shape:', embedded.shape)
        return self.fc(embedded)

In [189]:
glove_model = TextClassificationModel(pretrained_embeddings, len(classes)).to(device)

torch.Size([1193515, 200])


In [190]:
## OK let's try to run this model to see if dimensions are correct!


for idx, data in enumerate(train_dataset):
    print("---------------------------------")
    print('next example id ', idx)
    print("---------------------------------")
    example_text = data['text']
    print(example_text)
    print('token_len: ' , len(example_text))
    indices = glove_vocab(example_text)
    print('indices_len: ' , len(indices))
    text_input = torch.tensor(indices)
    print('tensor_shape: ', text_input.shape)
    model_output = glove_model(text_input)
    # fwd = glove_model.forward(text_input)
    # print('embedding_bag: ',  fwd.shape)
    print('output_shape: ', model_output.shape)

    #if idx == 2:
    break


## TODO: Modified the model with LSTM layers and train it.

---------------------------------
next example id  0
---------------------------------
['explanation', 'why', 'the', 'edits', 'made', 'under', 'my', 'username', 'hardcore', 'metallica', 'fan', 'were', 'reverted', '?', 'they', 'weren', "'", 't', 'vandalisms', ',', 'just', 'closure', 'on', 'some', 'gas', 'after', 'i', 'voted', 'at', 'new', 'york', 'dolls', 'fac', 'and', 'please', 'don', "'", 't', 'remove', 'the', 'template', 'from', 'the', 'talk', 'page', 'since', 'i', "'", 'm', 'retired', 'now', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>']
token_len:  100
indices_len:  100
tensor_shape:  torch.Size